<a href="https://colab.research.google.com/github/profteachkids/CHE4071_Fall2023/blob/main/dynamic_distillation4071.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import numpy as np
from plotly.subplots import make_subplots

In [14]:
x= np.linspace(0,0.999,50)
alpha = 5.

In [ ]:
tray_m = 0.1
reboiler_m = 0.5
condenser_m = 0.5

weir_constant = 0.1

F = 1.
z = 0.6
q = 0.6  #60mol% liquid

D = 0.55
R = 2.


V_rec = (R+1)*D  #Negligible vapor hold up, perfect control on reboiler boil up
V_strip = V_rec - (1-q)*F




m0 =
x0 =

In [ ]:
def rhs(t, vec):
    m, x = np.split(vec,2)
    y=x*alpha/(1-x + x*alpha)